In [ ]:
from langgraph.graph import StateGraph, START,END
from typing import TypedDict, Annotated
from langchain_core.messages import BaseMessage, HumanMessage
from langchain_google_genai import ChatGoogleGenerativeAI
from langgraph.checkpoint.memory import MemorySaver

In [ ]:
from langgraph.graph.message import add_messages

class ChatState(TypedDict):
    messages: Annotated[list[BaseMessage], add_messages]

In [ ]:
llm= ChatGoogleGenerativeAI(model='gemini-2.0-flash')

def chat_node(state: ChatState):
    messages= state['messages']
    
    response = llm.invoke(messages)
    
    return {'messages':[response]}

In [ ]:
checkpointer = MemorySaver()
graph = StateGraph(ChatState)

graph.add_node('chat_node',chat_node)

graph.add_edge(START,'chat_node')
graph.add_edge('chat_node',END)
chatbot=graph.compile(checkpointer=checkpointer)
chatbot

In [ ]:
initial_state = {
    'messages': [HumanMessage(content='what is graph theory?')]
    }
chatbot.invoke(initial_state)['messages'][-1].content

In [ ]:
thread_id = 1

while True:
    user_message = input('Type here:')
    print('User:',user_message)
    
    if user_message.strip().lower() in ['exit', 'quit','bye']:
        break
    
    config = {'configurable':{'thread_id':thread_id}}   
    
    response = chatbot.invoke({'messages':[HumanMessage(content=user_message)]},config=config)
    
    print('AI:', response['messages'][-1].content)